Clean and Merge Data
======================
This notebook loads and combines 3 different data files into a single dataframe. 
It combines _school demographic data_, _math test scortes_, and _ela test scores_.

For this example we try to use more re-usable functions to help clean our data.
In order to **merge** data sets, we need to have "keys" or "indices" in both
dataframes in order to know which rows to merge together. We know that our
schools have a unique key called `dbn` which we can use, but we also need to
combine it with the academic year in order to get a the correct unique id for each row.

- how to join datasets on columns
- how to rename columns


In [27]:
# load the demographic dataimport pandas as pd
import pandas as pd

demo_url = "https://data.cityofnewyork.us/resource/vmmu-wj3w.csv?$limit=1000000"
math_url = "https://data.cityofnewyork.us/resource/kha6-7i9i.csv?$limit=1000000"
ela_url = "https://data.cityofnewyork.us/resource/gu76-8i7h.csv?$limit=1000000"

### Load demgographics data
Here we load the school demographics data set from the API URL
and then clean it up a little bit.

We will merge this data with the test score data into a single data frame.

In [28]:
# load and prep the school demographics data
# generic function to take percents as Strings and convert to real
# expects data to look like '84.33%', 'Above 95%', 'Below 5%'
def str_pct(row, pct_col, enroll_col):
    pct = row[pct_col][:-1]
    # just call the population size `n`
    n = row[enroll_col]
    try:
        pct = float(pct) / 100
    except:
        if "Above" in pct:
            pct = n * .96 / n
        elif "Below" in pct:
            pct = n * .04 / n
    return float(pct)

# if the score isn't an int, set it to -1 to remove it or filter it easily
def clean_score(score):
    try:
        return int(score)
    except:
        return -1


# because `str_pct` is a generic function, we need to wrap it
# with another function, we can do this with `def` or `lambda`
# we'll do both, here
def pct_pov(row): return str_pct(row, "poverty_1", "total_enrollment")


# add district and boro info
boros = {"K":"Brooklyn", "X":"Bronx", "M": "Manhattan", "Q": "Queens", "R": "Staten Island"}

def district(dbn): return int(dbn[:2])
def boro(dbn): return boros[dbn[2]]

# convert the string format `year` to match the int format academic year
# used in the test score data
def ay(year): return int(year.split("-")[0])


# get the data and clean it a little bit
df = pd.read_csv(demo_url)

df["year"] = df["year"].apply(ay)
df["district"] = df["dbn"].apply(district)
df["boro"] = df["dbn"].apply(boro)
df["poverty_1"] = df.apply(pct_pov, axis = 1)
df["economic_need_index"] = df.apply(lambda row: str_pct(row, "economic_need_index", "total_enrollment"), axis = 1)


# drop the districts that aren't geographic districts b/c we don't have test data for them
df = df[df["district"] < 33]

# (optional) get just the columns we need, to make it more manageable

cols = ['dbn',
        'year',
        'district',
        'boro',
        'school_name', 
        'total_enrollment',
        'female_1',
        'male_1',
        'asian_1', 
        'black_1', 
        'hispanic_1', 
        'multi_racial_1', 
        'native_american_1', 
        'white_1', 
        'students_with_disabilities_1', 
        'english_language_learners_1',  
        'poverty_1',
        'economic_need_index']

# df = df[cols]
df.columns

Index(['dbn', 'school_name', 'year', 'total_enrollment',
       'grade_3k_pk_half_day_full', 'grade_k', 'grade_1', 'grade_2', 'grade_3',
       'grade_4', 'grade_5', 'grade_6', 'grade_7', 'grade_8', 'grade_9',
       'grade_10', 'grade_11', 'grade_12', 'female', 'female_1', 'male',
       'male_1', 'asian', 'asian_1', 'black', 'black_1', 'hispanic',
       'hispanic_1', 'multi_racial', 'multi_racial_1', 'native_american',
       'native_american_1', 'white', 'white_1', 'missing_race_ethnicity_data',
       'missing_race_ethnicity_data_1', 'students_with_disabilities',
       'students_with_disabilities_1', 'english_language_learners',
       'english_language_learners_1', 'poverty', 'poverty_1',
       'economic_need_index', 'district', 'boro'],
      dtype='object')

### Load and merge the math data
Here we get the math data and then fix up the columns. We rename the columns
so that they have the same names as the ELA columns we'll load later.

In [30]:
# now load the math data and prep it a little bit
math_df = pd.read_csv(math_url)

math_df["mean_scale_score"] = math_df["mean_scale_score"].apply(clean_score)
math_df.columns

Index(['dbn', 'school_name', 'grade', 'year', 'category', 'number_tested',
       'mean_scale_score', 'level1_n', 'level1_', 'level2_n', 'level2_',
       'level3_n', 'level3_', 'level4_n', 'level4_', 'level3_4_n',
       'level3_4_'],
      dtype='object')

In [31]:
# give the cols better names that align with the ela data
math_cols = {
  'level1_n': "level_1", # number of students score a 1
  'level1_': "level_1_1", # pct of students scoring a 1
  'level2_n': "level_2",
  'level2_': "level_2_1",
  'level3_n': "level_3",
  'level3_': "level_3_1",
  'level4_n': "level_4",
  'level4_': "level_4_1",
  'level3_4_n': "level_3_4",
  'level3_4_': "level_3_4_1"
}
 
# we will use only the school name from the demographics dataset
del math_df["school_name"]
math_df = math_df.rename(columns=math_cols)
math_df

,dbn,grade,year,category,number_tested,mean_scale_score,level_1,level_1_1,level_2,level_2_1,level_3,level_3_1,level_4,level_4_1,level_3_4,level_3_4_1
0,01M015,3,2013,All Students,27,278,16,59.3,11,40.7,0,0.0,0,0.0,0,0.0
1,01M015,3,2014,All Students,18,286,6,33.3,9,50.0,2,11.1,1,5.6,3,16.7
2,01M015,3,2015,All Students,17,280,10,58.8,4,23.5,2,11.8,1,5.9,3,17.6
3,01M015,3,2016,All Students,21,275,13,61.9,4,19.0,4,19.0,0,0.0,4,19.0
4,01M015,3,2017,All Students,29,302,8,27.6,9,31.0,7,24.1,5,17.2,12,41.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23891,32K562,All Grades,2013,All Students,145,266,107,73.8,35,24.1,3,2.1,0,0.0,3,2.1
23892,32K562,All Grades,2014,All Students,231,269,162,70.1,65,28.1,4,1.7,0,0.0,4,1.7
23893,32K562,All Grades,2015,All Students,324,268,249,76.9,70,21.6,4,1.2,1,0.3,5,1.5
23894,32K562,All Grades,2016,All Students,269,270,176,65.4,85,31.6,6,2.2,2,0.7,8,3.0


In [33]:
# merge the math data into our main data frame (now it's only the demo data)
# API documentation: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html

                    # data        # matching rows only       # matching cols
combined = df.merge(math_df,      how="inner",               on=["dbn", "year"])
cols = ["dbn","school_name", "total_enrollment", "white_1", "black_1", "hispanic_1", "asian_1", "mean_scale_score"]

combined[cols]

,dbn,school_name,total_enrollment,white_1,black_1,hispanic_1,asian_1,mean_scale_score
0,01M015,P.S. 015 Roberto Clemente,178,0.022,0.287,0.590,0.079,275
1,01M015,P.S. 015 Roberto Clemente,178,0.022,0.287,0.590,0.079,304
2,01M015,P.S. 015 Roberto Clemente,178,0.022,0.287,0.590,0.079,283
3,01M015,P.S. 015 Roberto Clemente,178,0.022,0.287,0.590,0.079,286
4,01M015,P.S. 015 Roberto Clemente,190,0.032,0.274,0.579,0.105,302
...,...,...,...,...,...,...,...,...
9400,32K562,Evergreen Middle School for Urban Exploration,325,0.018,0.258,0.711,0.006,270
9401,32K562,Evergreen Middle School for Urban Exploration,342,0.026,0.249,0.722,0.003,265
9402,32K562,Evergreen Middle School for Urban Exploration,342,0.026,0.249,0.722,0.003,275
9403,32K562,Evergreen Middle School for Urban Exploration,342,0.026,0.249,0.722,0.003,268


### Sorting schools on math performance
Now that we have the combined data, we can sort (and rank) the schools
based on their performance on the NYS math test and display
some of the demographic data for each school.

In [37]:
# which schools had the highest mean math score in 2019
high_math = combined.query("grade == 'All Grades' and year == 2017")

high_math = high_math.sort_values(by="mean_scale_score", ascending=False)

cols = ["dbn","school_name", "total_enrollment", "white_1", "black_1", "hispanic_1", "asian_1", "mean_scale_score"]
print("avg enrollment, top 100", high_math[cols][:100]["total_enrollment"].mean())

print("avg enrollment, bottom 100", high_math[cols][-100:]["total_enrollment"].mean())

high_math[cols]


avg enrollment, top 100 692.48
avg enrollment, bottom 100 396.08


,dbn,school_name,total_enrollment,white_1,black_1,hispanic_1,asian_1,mean_scale_score
5556,20K187,The Christa McAuliffe School\I.S. 187,873,0.239,0.013,0.068,0.675,373
8061,28Q303,The Academy for Excellence through the Arts,207,0.357,0.053,0.179,0.386,366
907,03M859,Special Music School,314,0.436,0.121,0.150,0.143,366
871,03M334,The Anderson School,537,0.512,0.028,0.091,0.270,365
948,04M012,Tag Young Scholars,587,0.153,0.186,0.170,0.388,365
...,...,...,...,...,...,...,...,...
1783,07X151,J.H.S. 151 Lou Gehrig,261,0.023,0.268,0.705,0.000,261
3678,12X314,Fairmont Neighborhood School,317,0.028,0.360,0.609,0.000,261
4185,14K586,Lyons Community School,496,0.026,0.371,0.593,0.010,259
7418,27Q053,M.S. 053 Brian Piccolo,267,0.015,0.521,0.431,0.011,258


### Load and merge ELA data
Merging the ELA data is similar to mergin the math data,
however we have to add a new key to our join -- grade level.
Also, because math and ela datasets share column names,
we specify `suffixes` when we call `merge()` on our dataframe.

In [38]:
# load the ela data from the URL
ela_df = pd.read_csv(ela_url)

ela_df["mean_scale_score"] = ela_df["mean_scale_score"].apply(clean_score)

# keep only the columns we want to merge
ela_df = ela_df[['dbn', 'grade', 'year', 'category',
       'number_tested', 'mean_scale_score', 'level_1', 'level_1_1', 'level_2',
       'level_2_1', 'level_3', 'level_3_1', 'level_4', 'level_4_1',
       'level_3_4', 'level_3_4_1']]


ela_df.columns

/tmp/ipykernel_20533/1592604141.py:2: DtypeWarning: Columns (7,8,9,10,11,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  ela_df = pd.read_csv(ela_url)


Index(['dbn', 'grade', 'year', 'category', 'number_tested', 'mean_scale_score',
       'level_1', 'level_1_1', 'level_2', 'level_2_1', 'level_3', 'level_3_1',
       'level_4', 'level_4_1', 'level_3_4', 'level_3_4_1'],
      dtype='object')

In [39]:
# now combine the ela stuff and see the new columns

ela_math_combo = combined.merge(ela_df, how="inner", on=["dbn", "year", "grade"], suffixes=["_math","_ela"])
ela_math_combo.columns


Index(['dbn', 'school_name', 'year', 'total_enrollment',
       'grade_3k_pk_half_day_full', 'grade_k', 'grade_1', 'grade_2', 'grade_3',
       'grade_4', 'grade_5', 'grade_6', 'grade_7', 'grade_8', 'grade_9',
       'grade_10', 'grade_11', 'grade_12', 'female', 'female_1', 'male',
       'male_1', 'asian', 'asian_1', 'black', 'black_1', 'hispanic',
       'hispanic_1', 'multi_racial', 'multi_racial_1', 'native_american',
       'native_american_1', 'white', 'white_1', 'missing_race_ethnicity_data',
       'missing_race_ethnicity_data_1', 'students_with_disabilities',
       'students_with_disabilities_1', 'english_language_learners',
       'english_language_learners_1', 'poverty', 'poverty_1',
       'economic_need_index', 'district', 'boro', 'grade', 'category_math',
       'number_tested_math', 'mean_scale_score_math', 'level_1_math',
       'level_1_1_math', 'level_2_math', 'level_2_1_math', 'level_3_math',
       'level_3_1_math', 'level_4_math', 'level_4_1_math', 'level_3_4_math

### Look at the combined data
Now that we have all three data sets merged, we can calculate a new column
that combines the Math and ELA scores for each school-year-grade observation.
Below we create the column, sort the data, and then show the results.

In [46]:
# add a new column with the combined average test score

cols = ["dbn","school_name", "total_enrollment", "white_1", "black_1", "hispanic_1", "asian_1", "avg_score"]


ela_math_combo["avg_score"] = (ela_math_combo["mean_scale_score_math"] + ela_math_combo["mean_scale_score_ela"]) / 2
avg = ela_math_combo.sort_values(by="avg_score", ascending=False)

avg[avg["avg_score"] > 0][cols]


,dbn,school_name,total_enrollment,white_1,black_1,hispanic_1,asian_1,avg_score
891,03M859,Special Music School,314,0.436,0.121,0.150,0.143,375.5
894,03M859,Special Music School,314,0.436,0.121,0.150,0.143,374.0
932,04M012,Tag Young Scholars,587,0.153,0.186,0.170,0.388,373.5
855,03M334,The Anderson School,537,0.512,0.028,0.091,0.270,369.0
8544,30Q122,P.S. 122 Mamie Fay,1366,0.411,0.059,0.212,0.272,369.0
...,...,...,...,...,...,...,...,...
148,01M363,Neighborhood School,304,0.461,0.092,0.349,0.063,159.0
1079,04M224,M.S. 224 Manhattan East School for Arts & Acad...,155,0.103,0.277,0.516,0.084,154.5
3627,12X267,Bronx Latin,579,0.012,0.190,0.777,0.014,152.5
3932,13K492,Academy of Arts and Letters,517,0.338,0.369,0.157,0.072,135.0
